<a id='home'></a>
### purpose

show environmental importance inferred from gradient forests

In [5]:
from pythonimports import *

training_dir = '/work/lotterhos/MVP-Offsets/practice_slim/training/gradient_forests/training_outfiles'

lview,dview = get_client()

latest_commit()
session_info.show()

36 36
##################################################################
Current commit of pythonimports:
commit 357f2a9069d9ca25062146953c9bf88b70e863c0  
Author: Brandon Lind <lind.brandon.m@gmail.com>  
Date:   Thu Feb 3 10:42:21 2022 -0500
Today:	February 17, 2022 - 15:45:09
python version: 3.8.5
##################################################################



In [3]:
# copy over script to extract climate importance from a trained Gradient Forest object
!cp /home/b.lind/code/GitHub/offset_validation/extract_importance.R .
!cat extract_importance.R


#-----------------------------------------------------------------------------------------------------#
# From a saved RDS object output from gradient forest training, extract predictor importance and save
#-----------------------------------------------------------------------------------------------------#

library(gradientForest)

args = commandArgs(trailingOnly=TRUE)
rdsfile = args[1]

rds = readRDS(rdsfile)

df1 = data.frame(overall_importance=rds$overall.imp)
df2 = data.frame(weighted_importance=importance(rds, type='Weighted'))
df3 = data.frame(accuracy_importance=importance(rds, type='Accuracy'))

# combine df1 with df2
df = merge(df1, df2, by='row.names', all=TRUE)
rownames(df) = df[,'Row.names']
df = subset(df, select=-c(Row.names))

# combine df1+df2 with df3
df = merge(df, df3, by='row.names', all=TRUE)
rownames(df) = df[,'Row.names']
df = subset(df, select=-c(Row.names))

outfile <- gsub(".RDS", "_importances.txt", rdsfile)

write.table(df, outfile, sep='	', row.names=TRU

In [15]:
script_file = fs(os.getcwd(), 'extract_importance.R')[0]
script_file

'/home/b.lind/code/MVP-offsets/00_explore_sims_output/extract_importance.R'

In [20]:
training_files = fs(training_dir, endswith='training.RDS')
[op.basename(f) for f in training_files]

['1231094_GF_training_ind_adaptive_gradient_forest_training.RDS',
 '1231094_GF_training_pooled_adaptive_gradient_forest_training.RDS',
 '1231094_GF_training_pooled_all_gradient_forest_training.RDS']

In [16]:
def extract_importance(f):
    """Extract importance values from RDS file, save to .txt file."""
    import subprocess
    
    output = subprocess.check_output(
        [
            Rscript,
            script_file,
            f
        ]
    ).decode("utf-8").replace("\n", "").split()[-1]
    
    return output

# prep for parallelization
Rscript = '/home/b.lind/anaconda3/envs/r35/bin/Rscript'
dview['Rscript'] = Rscript
dview['script_file'] = script_file


In [21]:
# read in importance values in parallel
jobs = []
for f in training_files:
    jobs.append(
        lview.apply_async(extract_importance, f)
    )

watch_async(jobs)


Watching 3 jobs ...


100%|███████████████| 3/3 [00:32<00:00, 10.81s/it]


In [22]:
# put parallel output into dictionary of ranks
importance_files = fs(training_dir, endswith='importances.txt')

importances = wrap_defaultdict(None, 4)
for f in pbar(importance_files):
    spp,method,setname,kfold = op.basename(f).rstrip("_gradient_forest_training_importances.txt").split('-')
    importances[spp][method][setname][kfold] = pd.read_table(f)

assert len(importance_files) == len(files)
    
importances[spp][method][setname][kfold].head()

  0%|               | 0/3 [00:00<?, ?it/s]


ValueError: not enough values to unpack (expected 4, got 1)

In [23]:
importance_files

['/work/lotterhos/MVP-Offsets/practice_slim/training/gradient_forests/training_outfiles/1231094_GF_training_ind_adaptive_gradient_forest_training_importances.txt',
 '/work/lotterhos/MVP-Offsets/practice_slim/training/gradient_forests/training_outfiles/1231094_GF_training_pooled_adaptive_gradient_forest_training_importances.txt',
 '/work/lotterhos/MVP-Offsets/practice_slim/training/gradient_forests/training_outfiles/1231094_GF_training_pooled_all_gradient_forest_training_importances.txt']

In [24]:
pd.read_table(importance_files[0])

,overall_importance,weighted_importance,accuracy_importance
sal_opt,0.002793,0.013922,0.002793
temp_opt,0.131401,0.307874,0.131401


In [25]:
[display(pd.read_table(f)) for f in importance_files]

,overall_importance,weighted_importance,accuracy_importance
sal_opt,0.002793,0.013922,0.002793
temp_opt,0.131401,0.307874,0.131401


,overall_importance,weighted_importance,accuracy_importance
sal_opt,-0.001759,0.003384,-0.001759
temp_opt,0.027816,0.578990,0.027816


,overall_importance,weighted_importance,accuracy_importance
sal_opt,-0.004800,0.001319,-0.004800
temp_opt,0.085967,0.684486,0.085967


[None, None, None]

In [26]:
[op.basename(f) for f in importance_files]

['1231094_GF_training_ind_adaptive_gradient_forest_training_importances.txt',
 '1231094_GF_training_pooled_adaptive_gradient_forest_training_importances.txt',
 '1231094_GF_training_pooled_all_gradient_forest_training_importances.txt']